In [1]:
import sqlite3

# 데이터베이스 파일 경로
db_path = r"C:\Users\Playdata\Desktop\만개레시피찐찐찐찐찐 최종\SKN06-FINAL-6Team\data\merged_jtbc.db"

# 데이터베이스 연결
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# 데이터베이스에 포함된 테이블 목록 확인
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# 테이블 목록 출력
tables


[('merged_result',)]

In [2]:
# 테이블의 구조(컬럼명 및 데이터 타입) 확인
cursor.execute("PRAGMA table_info(merged_result);")
table_info = cursor.fetchall()

# 컬럼명과 데이터 타입 정리
columns_info = [(col[1], col[2]) for col in table_info]
columns_info


[('id', 'INTEGER'),
 ('name', 'TEXT'),
 ('ingredients', 'TEXT'),
 ('recipes', 'TEXT'),
 ('img', 'TEXT'),
 ('video', 'TEXT')]

In [3]:
# 결측치 확인
import pandas as pd

# 데이터 불러오기
query = "SELECT * FROM merged_result"
df = pd.read_sql_query(query, conn)

# 결측치 개수 확인
missing_values = df.isnull().sum()
missing_values


id               0
name             0
ingredients      0
recipes          0
img            126
video          126
dtype: int64

In [4]:
# 결측치 대체
df["img"].fillna("No Image", inplace=True)
df["video"].fillna("No Video", inplace=True)

# 변경된 데이터 저장
df.to_sql("merged_result", conn, if_exists="replace", index=False)

# 변경 후 다시 결측치 확인
missing_values_after = df.isnull().sum()
missing_values_after

C:\Users\Playdata\AppData\Local\Temp\ipykernel_19108\1257795558.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["img"].fillna("No Image", inplace=True)
C:\Users\Playdata\AppData\Local\Temp\ipykernel_19108\1257795558.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

id             0
name           0
ingredients    0
recipes        0
img            0
video          0
dtype: int64

In [6]:
# 'name' 컬럼에서 요리사의 이름 뒤에 있는 '의'를 제거
df["name"] = df["name"].str.replace(r"(\S+)의 ", r"\1 ", regex=True)

# 변경된 데이터 저장
df.to_sql("merged_result", conn, if_exists="replace", index=False)

# 변경된 데이터 일부 확인
df["name"]


0             김성주 '연복풍 탕수육'
1             안정환 '테리우스테이크'
2      고든램지 '퀵! 차돌박이 볶음 말이'
3               레이먼킴 '팬타스틱'
4               김민준 '콩닭거렸쌈'
               ...         
989              최현석 'For덕'
990            미카엘 '프로틴 스틱'
991            홍석천 '불닭 소시지'
992     김풍 '매우면 참치 & 쏘루아밀크'
993         샘 킴 'Mr.콩chu~♥'
Name: name, Length: 994, dtype: object

In [7]:
# 'name' 컬럼에서 특정 요리사의 이름에 포함된 공백 제거
df["name"] = df["name"].str.replace("샘 킴", "샘킴").str.replace("레이먼 킴", "레이먼킴")

# 변경된 데이터 저장
df.to_sql("merged_result", conn, if_exists="replace", index=False)

# 변경된 데이터 일부 확인
df[df["name"].str.contains("샘킴|레이먼킴")].head()


,id,name,ingredients,recipes,img,video
3,4,레이먼킴 '팬타스틱',"재료아스파라거스, 베이컨, 소시지, 맥주, 팬케이크가루, 모차렐라치즈, 액상조미료,...","조리방법1. 아스파라거스를 적당한 크기로 손질한다.2. 양파, 베이컨을 적당한 크기...",https://fs.jtbc.co.kr/prog/enter/janggo/Img/20...,https://tv.jtbc.co.kr/vod/pr10010331/pm1003108...
125,126,레이먼킴 '조기 이즈 베리 임폴턴트',"재료조기, 마늘꿀, 녹차티백, 명란젓, 아몬드,잣, 크랜베리, 가지, 오리엔탈드레싱...",조리방법1. 조기는 머리를 잘라내고 지느러미와 가시를 발라 넓게 펼친다.2. 손질한...,https://fs.jtbc.co.kr/prog/enter/janggo/Img/20...,https://tv.jtbc.co.kr/vod/pr10010331/pm1003108...
126,127,레이먼킴 '1닭 2일',"재료치킨, 치킨무, 위스키, 토마토소스, 김,인스턴트 볶음김치, 통조림 햄, 양파,...","조리방법1. 양파, 마늘, 당근을 적당한 크기로 썬다.2. 기름을 두른 팬에 손질한...",https://fs.jtbc.co.kr/prog/enter/janggo/Img/20...,https://tv.jtbc.co.kr/vod/pr10010331/pm1003108...
127,128,레이먼킴 'HOT한 갈비찜',"재료인스턴트 소갈비탕, 새송이버섯, 콩나물,쌈장, 볶음고추장, 감자칩과자, 감자스틱...","조리방법1. 인스턴트 소갈비탕의 소갈비와 국물을 분리한다.2. 양파, 새송이버섯, ...",https://fs.jtbc.co.kr/joydata/CP00000001/prog/...,https://tv.jtbc.co.kr/vod/pr10010331/pm1003108...
128,129,레이먼킴 '갈치 한판승',"재료갈치, 말린 우엉, 머윗잎, 국물용 멸치, 인스턴트 재첩국, 국간장, 깻잎, 산...","조리 방법1. 국물백에 양파, 마늘, 국물용 멸치, 대파, 말린 우엉을 넣고 끓는 ...",https://fs.jtbc.co.kr/joydata/CP00000001/prog/...,https://tv.jtbc.co.kr/vod/pr10010331/pm1003108...


In [8]:
# 'ingredients' 컬럼에서 앞부분 '재료' 제거 및 특수문자 '●' 제거
df["ingredients"] = df["ingredients"].str.replace(r"^재료", "", regex=True).str.replace("●", "")

# 변경된 데이터 저장
df.to_sql("merged_result", conn, if_exists="replace", index=False)

# 변경된 데이터 일부 확인
df["ingredients"].head()


0    소고기 안심, 튀김가루, 적양배추, 대파, 마늘,고춧가루, 전분, 설탕, 식초, 간...
1    스테이크용 부챗살, 파프리카, 전복, 발사믹 식초,미역, 참기름, 양파, 마늘, 버...
2    알배추, 적양파, 청양고춧가루, 아스파라거스,불고기양념, 굴소스, 팽이버섯, 쪽파,...
3    아스파라거스, 베이컨, 소시지, 맥주, 팬케이크가루, 모차렐라치즈, 액상조미료,양파...
4    인스턴트 닭발, 송편, 곶감, 블루베리, 꿀에 절인 인삼, 인삼음료, 마누카 꿀, ...
Name: ingredients, dtype: object

In [9]:
# 'ingredients' 컬럼에서 ',' 뒤에 공백 추가하여 가독성 향상
df["ingredients"] = df["ingredients"].str.replace(",", ", ")

# 변경된 데이터 저장
df.to_sql("merged_result", conn, if_exists="replace", index=False)

# 변경된 데이터 일부 확인
df["ingredients"].head()


0    소고기 안심,  튀김가루,  적양배추,  대파,  마늘, 고춧가루,  전분,  설탕...
1    스테이크용 부챗살,  파프리카,  전복,  발사믹 식초, 미역,  참기름,  양파,...
2    알배추,  적양파,  청양고춧가루,  아스파라거스, 불고기양념,  굴소스,  팽이버...
3    아스파라거스,  베이컨,  소시지,  맥주,  팬케이크가루,  모차렐라치즈,  액상...
4    인스턴트 닭발,  송편,  곶감,  블루베리,  꿀에 절인 인삼,  인삼음료,  마...
Name: ingredients, dtype: object

In [10]:
# 'recipes' 컬럼에서 앞부분 '조리방법' 제거
df["recipes"] = df["recipes"].str.replace(r"^조리방법", "", regex=True)

# 변경된 데이터 저장
df.to_sql("merged_result", conn, if_exists="replace", index=False)

# 변경된 데이터 일부 확인
df["recipes"].head()


0    1. 끓는 기름에 적당한 크기로 썬 대파와 으깬 마늘, 고춧가루를 넣고 고추기름을 ...
1    1. 양파를 잘게 썬다.2. 파프리카를 직화한다.3. 마늘은 편으로 썰고, 전복은 ...
2    1. 알배추, 적양파를 채 썬다.2. 올리브오일과 참기름을 두른 팬에 채 썬 적양파...
3    1. 아스파라거스를 적당한 크기로 손질한다.2. 양파, 베이컨을 적당한 크기로 썰어...
4    1. 냄비에 인삼음료를 넣고 끓인다.2. 끓이던 인삼음료에 설탕, 물, 물엿, 마누...
Name: recipes, dtype: object

In [11]:
# 'recipes' 컬럼에서 특수문자 '①, ②, ③...' 등을 '1., 2., 3.' 형태로 변경
df["recipes"] = df["recipes"].str.replace("①", "1. ").str.replace("②", "2. ").str.replace("③", "3. ") \
                             .str.replace("④", "4. ").str.replace("⑤", "5. ").str.replace("⑥", "6. ") \
                             .str.replace("⑦", "7. ").str.replace("⑧", "8. ").str.replace("⑨", "9. ") \
                             .str.replace("⑩", "10. ")

# 변경된 데이터 저장
df.to_sql("merged_result", conn, if_exists="replace", index=False)

# 변경된 데이터 일부 확인
df["recipes"].head()


0    1. 끓는 기름에 적당한 크기로 썬 대파와 으깬 마늘, 고춧가루를 넣고 고추기름을 ...
1    1. 양파를 잘게 썬다.2. 파프리카를 직화한다.3. 마늘은 편으로 썰고, 전복은 ...
2    1. 알배추, 적양파를 채 썬다.2. 올리브오일과 참기름을 두른 팬에 채 썬 적양파...
3    1. 아스파라거스를 적당한 크기로 손질한다.2. 양파, 베이컨을 적당한 크기로 썰어...
4    1. 냄비에 인삼음료를 넣고 끓인다.2. 끓이던 인삼음료에 설탕, 물, 물엿, 마누...
Name: recipes, dtype: object

In [ ]:
def preprocess_ingredients(df):
    """
    주어진 데이터프레임(df)의 'ingredients' 컬럼을 리스트로 변환하는 전처리 함수.

    - 쉼표(,)를 기준으로 재료를 분리
    - 공백을 제거하여 정확한 검색 가능하도록 정리

    Parameters:
        df (pd.DataFrame): 레시피 데이터프레임

    Returns:
        pd.DataFrame: 전처리된 데이터프레임
    """
    df = df.copy()  # 원본 데이터 변경 방지
    
    df["ingredients_list"] = df["ingredients"].apply(lambda x: [i.strip() for i in str(x).split(",")])
    
    return df

In [ ]:
# 전처리된 데이터를 새로운 SQLite DB 파일로 저장하는 코드

# 새로운 데이터베이스 파일 경로 설정
new_db_path = r"C:\Users\Playdata\Desktop\만개레시피찐찐찐찐찐 최종\SKN06-FINAL-6Team\data\merged_jtbc_1차.db"

# SQLite 데이터베이스에 저장
conn_new = sqlite3.connect(new_db_path)
df.to_sql("funstaurant_cleaned", conn_new, if_exists="replace", index=False)
conn_new.close()

# 저장된 DB 파일 경로 출력
new_db_path
